$$
Modelo
$$

In [1]:
# importar librerias necesarias
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

In [2]:

#importar datos
df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id','title'])
df_scores = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId','score','movieId'])
df_movie = df_movie.reset_index(drop=True)



### visualizar solo las columnas que usare para el modelo y la recomendacion

In [3]:
# visualizar peliculas
df_movie.head()

,id,title
0,ns1,dick johnson is dead
1,ns2,blood & water
2,ns3,ganglands
3,ns4,jailbirds new orleans
4,ns5,kota factory


In [4]:
# visualizar los ratings de peliculas por usuario
df_scores.head()

,userId,score,movieId
0,1,1.0,as680
1,1,4.5,ns2186
2,1,5.0,hs2381
3,1,5.0,ns3663
4,1,5.0,as9500


In [7]:
# Crear un objeto Reader para analizar los datos de entrada
reader = Reader(rating_scale=(0.5, 5.0))


### elegir un numero inferior al dataset paar entrenar el modelo y asi evitar el overfiting

In [8]:
limite = 205000
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df_scores[['userId', 'movieId','score']][:limite], reader)

### Entrenamento de modelo

In [9]:
# Dividir los datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.3, random_state=42)

# Entrenar un modelo SVD
model = SVD(n_factors=50, lr_all=0.005, reg_all=0.02, n_epochs=100)
model.fit(trainset)

In [10]:
# Hacer predicciones para un usuario específico
user_id = 1
movies_watched_by_user = df_scores[df_scores['userId'] == user_id]['movieId'].tolist()
unseen_movies_by_user = df_scores[~df_scores['movieId'].isin(movies_watched_by_user)]['movieId'].tolist()
testset = [[user_id, movie_id, 4] for movie_id in unseen_movies_by_user]
predictions = model.test(testset)

In [11]:
# Obtener las 5 películas más recomendadas
top_n = 5
recommended_movies = []
for uid, iid, _, est, _ in predictions:
    recommended_movies.append((iid, est))
recommended_movies.sort(key=lambda x: x[1], reverse=True)
recommended_movies = recommended_movies[:top_n]

In [12]:
# Obtener los títulos de las películas recomendadas
recommended_movies_titles = df_movie[df_movie['id'].isin([x[0] for x in recommended_movies])]['title'].tolist()

print('Las películas recomendadas para el usuario {} son:'.format(user_id))
for i, movie_title in enumerate(recommended_movies_titles):
    print('{}. {}'.format(i+1, movie_title))


Las películas recomendadas para el usuario 1 son:
1. footprints: the path of your life
2. melrose place
3. the simpsons
4. little baby bus and friends
5. cruisin' route 66 with michael wallis


mejorado

In [13]:
# importar librerias necesarias
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

# importar datos
df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id','title'])
df_scores = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId','score','movieId'])
df_movie = df_movie.reset_index(drop=True)

# Crear un objeto Reader para analizar los datos de entrada
reader = Reader(rating_scale=(1, 5))

limite = 205000
data = Dataset.load_from_df(df_scores[['userId', 'movieId', 'score']][:limite], reader)

# Dividir los datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.3, random_state=42)

# Entrenar un modelo SVD
model = SVD(n_factors=50, lr_all=0.005, reg_all=0.02, n_epochs=100)
model.fit(trainset)

# Hacer predicciones para un usuario específico
user_id = 1
movies_watched_by_user = df_scores[df_scores['userId'] == user_id]['movieId'].tolist()
unseen_movies_by_user = df_scores[~df_scores['movieId'].isin(movies_watched_by_user)]['movieId'].tolist()
testset = [[user_id, movie_id, 4] for movie_id in unseen_movies_by_user]
predictions = model.test(testset)

# Obtener las películas recomendadas
recommended_movies = [(pred.iid, pred.est) for pred in predictions]
recommended_movies.sort(key=lambda x: x[1], reverse=True)
recommended_movie_ids = [movie[0] for movie in recommended_movies]

# Obtener los títulos de las películas recomendadas
recommended_movies_titles = df_movie[df_movie['id'].isin(recommended_movie_ids)]['title'].tolist()

print('Las películas recomendadas para el usuario {} son:'.format(user_id))
for i, movie_title in enumerate(recommended_movies_titles):
    print('{}. {}'.format(i + 1, movie_title))


modelo

In [1]:
# funcion de recomendacion
# importar librerias necesarias
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import pandas as pd

def get_recommended_movies(user_id):
    # importar datos
    df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id','title'])
    df_scores = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId','score','movieId'])
    df_movie = df_movie.reset_index(drop=True)

    # Crear un objeto Reader para analizar los datos de entrada
    reader = Reader(rating_scale=(1, 5))

    limite = 205000
    data = Dataset.load_from_df(df_scores[['userId', 'movieId', 'score']][:limite], reader)

    # Dividir los datos en conjuntos de entrenamiento y prueba
    trainset, testset = train_test_split(data, test_size=0.3, random_state=42)

    # Entrenar un modelo SVD
    model = SVD(n_factors=50, lr_all=0.005, reg_all=0.02, n_epochs=100)
    model.fit(trainset)

    # Hacer predicciones para el usuario específico
    movies_watched_by_user = df_scores[df_scores['userId'] == user_id]['movieId'].tolist()
    unseen_movies_by_user = df_scores[~df_scores['movieId'].isin(movies_watched_by_user)]['movieId'].tolist()
    testset = [[user_id, movie_id, 4] for movie_id in unseen_movies_by_user]
    predictions = model.test(testset)

    # Obtener las películas recomendadas
    recommended_movies = [(pred.iid, pred.est) for pred in predictions]
    recommended_movies.sort(key=lambda x: x[1], reverse=True)
    recommended_movie_ids = [movie[0] for movie in recommended_movies]

    # Obtener los títulos de las películas recomendadas
    recommended_movies_titles = df_movie[df_movie['id'].isin(recommended_movie_ids)]['title'].tolist()

    return recommended_movies_titles


interface

In [2]:
import gradio as gr
from getpass import getpass

# Definir la función de clasificación
def recommend_movies(user_id):
    recommended_movies = get_recommended_movies(user_id)
    return "\n".join(recommended_movies)

# Definir la interfaz de usuario
user_id_input = gr.inputs.Textbox(label="User ID")

interface = gr.Interface(
    fn=recommend_movies,
    inputs=user_id_input,
    outputs="text",
    title="Movie Recommender",
    description="Enter a user ID to get recommended movies."
)

# Ejecutar la interfaz de usuario
interface.launch()


d:\MisProyectos\ApiStream\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\MisProyectos\ApiStream\venv\Lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
d:\MisProyectos\ApiStream\venv\Lib\site-packages\gradio\inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
d:\MisProyectos\ApiStream\venv\Lib\site-packages\gradio\inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


funcion entrena el modelo

In [3]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import pandas as pd

# Función para entrenar el modelo y devolverlo
def train_model():
    # Importar datos
    df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id','title'])
    df_scores = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId','score','movieId'])
    df_movie = df_movie.reset_index(drop=True)

    # Crear un objeto Reader para analizar los datos de entrada
    reader = Reader(rating_scale=(1, 5))

    limite = 205000
    data = Dataset.load_from_df(df_scores[['userId', 'movieId', 'score']][:limite], reader)

    # Dividir los datos en conjuntos de entrenamiento y prueba
    trainset, testset = train_test_split(data, test_size=0.3, random_state=42)

    # Entrenar un modelo SVD
    model = SVD(n_factors=50, lr_all=0.005, reg_all=0.02, n_epochs=100)
    model.fit(trainset)

    return model

In [4]:
# Función para realizar las predicciones con el modelo entrenado
def predict_movies(model, user_id):
    # Importar datos
    df_movie = pd.read_csv("../datos/csvs/peliculas/peliculas_final.csv", sep=',', usecols=['id','title'])
    df_scores = pd.read_csv("../datos/csvs/ratings/ratings_final.csv", sep=',', usecols=['userId','score','movieId'])
    df_movie = df_movie.reset_index(drop=True)

    # Hacer predicciones para el usuario específico
    movies_watched_by_user = df_scores[df_scores['userId'] == user_id]['movieId'].tolist()
    unseen_movies_by_user = df_scores[~df_scores['movieId'].isin(movies_watched_by_user)]['movieId'].tolist()
    testset = [[user_id, movie_id, 4] for movie_id in unseen_movies_by_user]
    predictions = model.test(testset)

    # Obtener las películas recomendadas
    recommended_movies = [(pred.iid, pred.est) for pred in predictions]
    recommended_movies.sort(key=lambda x: x[1], reverse=True)
    recommended_movie_ids = [movie[0] for movie in recommended_movies]

    # Obtener los títulos de las películas recomendadas
    recommended_movies_titles = df_movie[df_movie['id'].isin(recommended_movie_ids)]['title'].tolist()

    return recommended_movies_titles

# Entrenar el modelo
trained_model = train_model()

In [5]:
import gradio as gr
# Definir la función de clasificación para Gradio
def recommend_movies(user_id):
    recommended_movies = predict_movies(trained_model, user_id)
    return "\n".join(recommended_movies)

# Definir la interfaz de usuario
user_id_input = gr.inputs.Textbox(label="User ID")

interface = gr.Interface(
    fn=recommend_movies,
    inputs=user_id_input,
    outputs="text",
    title="Movie Recommender",
    description="Enter a user ID to get recommended movies."
)

# Ejecutar la interfaz de usuario
interface.launch()

d:\MisProyectos\ApiStream\venv\Lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
d:\MisProyectos\ApiStream\venv\Lib\site-packages\gradio\inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
d:\MisProyectos\ApiStream\venv\Lib\site-packages\gradio\inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


El código proporcionado es una implementación de un sistema de recomendación de películas utilizando el algoritmo de Factorización de Matrices Singular Value Decomposition (SVD) de la biblioteca Surprise. El sistema permite ingresar un ID de usuario y devuelve una lista de películas recomendadas para ese usuario en función de los datos de películas y puntajes proporcionados.

El funcionamiento del código se puede describir en los siguientes pasos:

1. Se importan las librerías necesarias, incluyendo Gradio, la biblioteca Surprise y pandas para el manejo de datos.

2. La función `train_model()` se encarga de cargar los datos de películas y puntajes, crear un objeto `Reader` para analizar los datos de entrada, dividir los datos en conjuntos de entrenamiento y prueba, y entrenar un modelo SVD utilizando el conjunto de entrenamiento. Luego, devuelve el modelo entrenado.

3. La función `predict_movies(model, user_id)` toma el modelo entrenado y un ID de usuario como entrada. Utiliza el modelo para hacer predicciones de películas para el usuario específico. Primero, obtiene las películas que el usuario ya ha visto y las películas que el usuario aún no ha visto. Luego, genera un conjunto de prueba con el usuario, las películas no vistas y una puntuación inicial de 4. A continuación, hace predicciones utilizando el modelo para este conjunto de prueba. Las predicciones contienen el ID de la película y la estimación de puntuación. Luego, ordena las predicciones en orden descendente según la estimación de puntuación y obtiene los IDs de las películas recomendadas. Finalmente, obtiene los títulos de las películas recomendadas utilizando los IDs y los devuelve en forma de lista.

4. Se entrena el modelo llamando a la función `train_model()`, y se guarda en la variable `trained_model`.

5. Se define la función `recommend_movies(user_id)` como la función de clasificación para Gradio. Toma un ID de usuario como entrada y llama a la función `predict_movies()` con el modelo entrenado y el ID de usuario para obtener la lista de películas recomendadas. Luego, devuelve las películas recomendadas como una cadena de texto con saltos de línea entre cada título.

6. Se define la interfaz de usuario utilizando Gradio. Se utiliza un cuadro de texto para ingresar el ID de usuario. La función de clasificación `recommend_movies()` se asigna como la función a ejecutar cuando se envía el formulario.

7. Se lanza la interfaz de usuario utilizando `interface.launch()`, lo que permite interactuar con la aplicación y obtener las películas recomendadas para un usuario ingresado.

Con esta implementación, los usuarios pueden ingresar su ID y obtener recomendaciones de películas personalizadas basadas en el modelo SVD entrenado.